# Registration
This script will creat "reg" directory in all bold directories and get registration infromation from functional space to anatomical space and from anatomical space to standard space. At the end, it will concatinate the results to have transformation from funrtional space to standard space.

Linear registration:      functional space >>> anatomical space
Non-Lineare registration: anatomical space >>> standard space

-----------------------------------------------------------
Script written by Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

## Anatomical >> Standard space registration

In [ ]:
import glob
import os


data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'
reference_path = '/mnt/d/Data/Pigeon/Sleep/Scripts/template/'
sub_dirs = glob.glob('%s/sub*/20*'%(data_path))

for curr_sub in sub_dirs:
    
    reg_directory = os.path.join(curr_sub, 'results','reg')
    if os.path.isdir(reg_directory):
        print('%s directory exist! Deleting the directory ...'%(reg_directory))
        os.system('rm -r %s'%(reg_directory))
        
        print('Creating directory %s ...'%(reg_directory))
        os.makedirs(reg_directory)

    else:
        print('Creating directory %s ...'%(reg_directory))
        os.makedirs(reg_directory)

    print('\t 1)Copying the t2w and standard images to %s ...'%(reg_directory))

    #os.system('fslmaths %s/t2w.nii.gz %s/highres'%(curr_sub,reg_directory))
    os.system('fslmaths %s/t2w.nii.gz %s/highres_head'%(curr_sub,reg_directory))
    os.system('fslmaths %s/standard_brain.nii.gz %s/standard'%(reference_path,reg_directory))
    os.system('fslmaths %s/standard_head.nii.gz %s/standard_head'%(reference_path,reg_directory))
    os.system('fslmaths %s/standard_brain_mask.nii.gz %s/standard_mask'%(reference_path,reg_directory))
    
    # change directory to the reg 
    os.chdir(reg_directory)
    
    #highres 2 standard
    # if you want to use FSL nonlinear registeration
    print('\t 2) Linear registration of anatomical image to the standard space ...')
    os.system('flirt -in highres_head -ref standard_head -out highres2standard -omat highres2standard.mat -cost corratio -dof 12 -searchrx -180 180 -searchry -180 180 -searchrz -180 180 -interp trilinear')
    
    print('\t 3) Non-Linear registration of anatomical image to the standard space ... \n')
    os.system('fnirt --in=highres_head --aff=highres2standard.mat --cout=highres2standard_warp --iout=highres2standard --jout=highres2highres_jac --ref=standard_head --refmask=standard_mask --warpres=6,6,6 --regmod=membrane_energy')
    os.system('applywarp -i highres_head -r standard_head -o highres2standard -w highres2standard_warp')
    
    #converting the transformation matrix from standard space to anatomical space
    os.system('convert_xfm -inverse -omat standard2highres.mat highres2standard.mat')
    
    # creating .png file to demonstrate the quality of registration 
    os.system('slicer highres2standard standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png highres2standard1.png ; slicer standard highres2standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png highres2standard2.png ; pngappend highres2standard1.png - highres2standard2.png highres2standard.png; rm -f sl?.png highres2standard2.png')
    
    # removing the non related files
    os.system('rm highres2standard1.png')

### Plot Anatomical >> Atlas registration results

In [ ]:
%matplotlib inline
%pylab inline
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from IPython.display import display, Image
import glob
import os

data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'
reg_dirs = glob.glob('%s/sub*/*/results/reg'%(data_path))

for curr_reg in reg_dirs:
    
    #print(curr_reg)
    print(curr_reg.split('/')[-4:-2])
    plt.show()
    display(Image(filename=os.path.join(curr_reg, 'highres2standard.png' )))

## Creat example_func and save in reg directory

In [ ]:
#middle volume save as example_func
import glob
import os
import subprocess

data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'
bold_dirs = glob.glob('%s/sub*/*/bold.nii.gz'%(data_path))
dummy_scan = 5
for curr_bold in bold_dirs:
    print('Current file is %s \n'%(curr_bold))
    curr_dir = os.path.dirname(curr_bold)
    reg_dst = os.path.join(curr_dir,'results','reg')
    
    # create example_func file: the midle volume of session
    nVloume = subprocess.check_output("fslnvols %s"%(curr_bold), shell=True).decode('utf-8')
    os.system("fslroi %s %s/example_func %s %s"%(curr_bold,reg_dst,round((int(nVloume))/2+dummy_scan),1))

## Linear registration of Functional data into the Anatomical space 

You must check the results to make sure the registration is done in correct way.


In [ ]:
import glob
import os
import subprocess
data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'
reg_dirs = glob.glob('%s/sub*/*/results/reg'%(data_path))
for curr_reg in reg_dirs:
    os.chdir(curr_reg)
    print('Current folder is %s  \n'%(curr_reg))

    #FSL
    # register func2highres
    os.system('flirt -in example_func -ref highres_head -out example_func2highres -omat example_func2highres.mat -bins 256 -cost corratio -searchrx -180 180 -searchry -180 180 -searchrz -180 180 -dof 12  -interp trilinear')
    os.system('convert_xfm -inverse -omat highres2example_func.mat example_func2highres.mat')
    os.system('slicer example_func2highres highres_head -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2highres1.png ; slicer highres_head example_func2highres -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2highres2.png ; pngappend example_func2highres1.png - example_func2highres2.png example_func2highres.png; rm -f sl?.png example_func2highres2.png')
    os.system('rm example_func2highres1.png')


### Plot example_func >> anatomical space registeration results

In [ ]:
%matplotlib inline
%pylab inline
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from IPython.display import display, Image
import glob
import os

data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'

reg_dirs = glob.glob('%s/sub*/*/results/reg'%(data_path))
for curr_reg in reg_dirs:
    
    print(curr_reg.split('/')[-4], curr_reg.split('/')[-3])
    plt.show()
    display(Image(filename=os.path.join(curr_reg, 'example_func2highres.png' )))


## concatenate functional >> highres and highres >> standrad

In [ ]:
import glob
import os
import subprocess
data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'

reg_dirs = glob.glob('%s/sub*/*/results/reg'%(data_path))

for curr_reg in reg_dirs:
    print('1) Concatenating transfomrations from folder: %s'%(curr_reg))
    # change path
    os.chdir(curr_reg)
    
    # convert tranform matrix 
    os.system('convert_xfm -omat example_func2standard.mat -concat highres2standard.mat example_func2highres.mat')
    os.system('convertwarp --ref=standard --premat=example_func2highres.mat --warp1=highres2standard_warp --out=example_func2standard_warp')
    os.system('applywarp --ref=standard --in=example_func --out=example_func2standard --warp=example_func2standard_warp')
    os.system('slicer example_func2standard standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2standard1.png ; /usr/local/fsl/bin/slicer standard example_func2standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2standard2.png ; /usr/local/fsl/bin/pngappend example_func2standard1.png - example_func2standard2.png example_func2standard.png; /bin/rm -f sl?.png example_func2standard2.png')
    
    print('2) Reversng example_func2standard Warp to standard2example_func Warp ... \n\n')
    os.system('convert_xfm -inverse -omat standard2example_func.mat example_func2standard.mat')
    os.system('invwarp --ref=example_func --warp=example_func2standard_warp --out=standard2example_func_warp')
    os.system('applywarp --ref=example_func --in=standard --out=standard2example_func --warp=standard2example_func_warp')
    os.system('slicer standard2example_func example_func -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png standard2example_func1.png ; /usr/local/fsl/bin/slicer example_func standard2example_func -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png standard2example_func2.png ; /usr/local/fsl/bin/pngappend standard2example_func1.png - standard2example_func2.png standard2example_func.png; /bin/rm -f sl?.png standard2example_func2.png')
    os.system('fslmaths standard2example_func -bin example_func_mask')
    #os.system('rm -r *1.png')

# Plot standard >> functional space results
You should make sure that the reversing the mask works properly.

In [ ]:
%matplotlib inline
%pylab inline
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from IPython.display import display, Image
import glob
import os

data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'

reg_dirs = glob.glob('%s/sub*/*/results/reg'%(data_path))
for curr_reg in reg_dirs:
    
    print(curr_reg.split('/')[-4], curr_reg.split('/')[-3])
    plt.show()
    display(Image(filename=os.path.join(curr_reg, 'standard2example_func.png' )))

# Converting example_func mask to anatomical space

In [ ]:
import glob
import os

data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'

reg_dirs = glob.glob('%s/sub*/*/results/reg'%(data_path))

for curr_reg in reg_dirs:
    
    print('Current folder is: %s \n'%(curr_reg))
    # change path
    os.chdir(curr_reg)
    os.system('flirt -in example_func_mask.nii.gz -ref highres_head -applyxfm -init example_func2highres.mat -interp nearestneighbour -out highres_mask')
    os.system('fslmaths highres_head -mul highres_mask highres')


# plot mask in functional spaces

In [ ]:
import glob
import os
import pandas as pd
from nilearn.image import new_img_like, load_img, get_data
import numpy as np
from nilearn import plotting

data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'

reg_dirs = glob.glob('%s/sub*/*/results/reg'%(data_path))

for curr_reg in reg_dirs:
        
    print(curr_reg.split('/')[-3],curr_reg.split('/')[-2])
    mask_img = load_img(os.path.join(curr_reg, 'example_func_mask.nii.gz'))
    # .astype() makes a copy.
    process_mask = get_data(mask_img).astype(np.int)
    
    process_mask_img = new_img_like(mask_img, process_mask)

    plotting.plot_roi(process_mask_img, bg_img=os.path.join(curr_reg, 'example_func.nii.gz'),
                          cmap='Paired')
    plotting.show()


# Plot example_func >> standard space registeration results

In [ ]:
%matplotlib inline
%pylab inline
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from IPython.display import display, Image
import glob
import os

data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'

reg_dirs = glob.glob('%s/sub*/*/results/reg'%(data_path))
for curr_reg in reg_dirs:
        
    print(curr_reg.split('/')[-4],curr_reg.split('/')[-3])
    plt.show()
    display(Image(filename=os.path.join(curr_reg, 'example_func2standard.png' )))
